In [1]:
import numpy as np

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from scipy import stats

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

import time
from datetime import datetime

from collections import defaultdict
from collections import Counter

from random import randint
import pickle

import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings('error')

import string

import json

from mosestokenizer import *

In [2]:
with open('foilGWVQA.train.json', 'r') as f:
    train = json.load(f)

with open('foilGWVQA.val.json', 'r') as f:
    valid = json.load(f)
    
with open('foilGWVQA.test.json', 'r') as f:
    test = json.load(f)

In [4]:
def get_tokenized_sentence(sentence, token_set):
    
    tokenized_sentence = []
    
    with MosesTokenizer('en') as tokenize:
            
        sentence = sentence.replace('\n', '')
        sentence = [t.lower() for t in tokenize(sentence)]
        
        for s in sentence:
            token_set.add(s)
            
    return sentence, token_set


# train_set = []

# portion = len(train['annotations'])

# tokens = set()

# with MosesTokenizer('en') as tokenizer:

#     for i in range(portion):

#         t = train['annotations'][i]

#         tokenized_caption, tokens = get_tokenized_sentence(t['caption'], tokens)
                        
#         if t['foilWord'] == 'Original':
#             caption_class = 0

#         else:
#             caption_class = 1
            
#         train_set.append((tokenized_caption, caption_class))
#         if i %100 == 0:
#             print(i)
            
            
valid_set = []

portion = len(valid['annotations'])

with MosesTokenizer('en') as tokenizer:

    for i in range(portion):

        t = valid['annotations'][i]

        tokenized_caption, tokens = get_tokenized_sentence(t['caption'], tokens)
                        
        if t['foilWord'] == 'Original':
            caption_class = 0

        else:
            caption_class = 1
            
        valid_set.append((tokenized_caption, caption_class))
        if i %100 == 0:
            print(i)
            
# test_set = []

# portion = len(test['annotations'])


# with MosesTokenizer('en') as tokenizer:

#     for i in range(portion):

#         t = test['annotations'][i]

#         tokenized_caption, tokens = get_tokenized_sentence(t['caption'], tokens)
                        
#         if t['foilWord'] == 'Original':
#             caption_class = 0

#         else:
#             caption_class = 1
            
#         test_set.append((tokenized_caption, caption_class))
#         if i %100 == 0:
#             print(i)
            

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500


In [5]:
with open('tokenized_annotations_train.json', 'w') as file:
    json.dump(train_set, file)
    
with open('tokenized_annotations_valid.json', 'w') as file:
    json.dump(valid_set, file)

with open('tokenized_annotations_test.json', 'w') as file:
    json.dump(test_set, file)

NameError: name 'test_set' is not defined

In [7]:
len(valid_set)

5596

In [65]:
with open('tokenized_annotations_train.json', 'r') as file:
    train_set = json.load(file)
    
with open('tokenized_annotations_valid.json', 'r') as file:
    valid_set = json.load(file)

with open('tokenized_annotations_test.json', 'r') as file:
    test_set = json.load(file)

In [58]:
tokens = set()

for i in range(len(train_set)):

    t_split = train_set[i][0]
    
    for t in t_split:
        tokens.add(t)
        

for i in range(len(test_set)):

    t_split = test_set[i][0]
    
    for t in t_split:
        tokens.add(t)
        

for i in range(len(valid_set)):

    t_split = valid_set[i][0]
    
    for t in t_split:
        tokens.add(t)

In [59]:
token2id = defaultdict(int)
count = 0
for t in tokens:
    token2id[t] = count
    count += 1

In [60]:
def get_sent2id(sentence):
    
    sentence2id = []
    
    for s in sentence:
        sentence2id.append(token2id[s])
        
    return sentence2id

In [61]:
with open('token2id.json', 'w') as file:
    json.dump(token2id, file)

In [66]:
with open('token2id.json', 'r') as file:
    token2id = json.load(file)

In [67]:
token2id['taken']

2596

In [80]:
class BlindLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(BlindLSTM, self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        self.embeddings = nn.Embedding(self.vocab_size, self.embedding_dim)
        
        self.LSTM = nn.LSTM(self.embedding_dim, self.embedding_dim, bidirectional=False)
        
        self.affine = nn.Linear(self.embedding_dim, 2)
        
        
            
    def init_hidden(self):
        return(Variable(torch.zeros(1,1,self.embedding_dim)), Variable(torch.zeros(1,1,self.embedding_dim, requires_grad=True)))
        
    def forward(self, sentence, hidden):

        embeddings = self.embeddings(sentence)

        output, new_hidden = self.LSTM(embeddings.view(len(sentence), 1, -1), hidden)

        xi = self.affine(output[-1])
        xi = F.relu(xi)
        xi = F.softmax(xi, dim = 1)
        return xi, new_hidden
    

epochs = 100
learning_rate = 0.01
portion = 100 #len(train_set)
embedding_dim = 100
vocab_size = len(token2id)
#batch_size = 100

model = BlindLSTM(vocab_size, embedding_dim)
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

losses = []
avg_losses = []

print('epoch, total loss, average loss, duration')
for e in range(epochs):
    
    then = datetime.now()
    
    total_loss = 0.0
    
    for b in range(portion):

        sent2id = get_sent2id(train_set[b][0])
        sentence = Variable(torch.tensor(np.asarray(sent2id), dtype= torch.long))
        class_sent = Variable(torch.tensor(np.asarray(train_set[b][1]), dtype= torch.long).view(1))

        soft_output, new_hidden = model(sentence, model.init_hidden())

        optimizer.zero_grad()

        loss = loss_fn(soft_output.view(1,-1), class_sent)
        loss.backward()

        optimizer.step()

        total_loss += loss.item() 
    
    
    now = datetime.now()
        
    losses.append(total_loss)
    
    avg_loss = np.mean(losses)/portion
    
    print(e, total_loss, avg_loss, now-then)
    
    avg_losses.append(avg_loss)
    
with open('model_blind_lstm' + str(portion) + '.pickle','wb') as file:
    pickle.dump(model,file)
      
iteration= list(range(len(avg_losses)))

plt.plot(iteration, avg_losses)
plt.xlabel("Iterations for Blind LSTM")
plt.ylabel('Average loss')
plt.title('Evolution of the loss as a function of the iteration')
plt.savefig("Blind LSTM" + str(portion)+".png")
plt.show()

with torch.no_grad():
    sent = train_set[0][0]
    class_sent = train_set[0][1]
    
    sent2id = get_sent2id(sent)
    sentence = torch.tensor(np.asarray(sent2id), dtype= torch.long)
    class_sent = torch.tensor(np.asarray(class_sent), dtype= torch.long).view(1)

    soft, hidden = model(sentence, model.init_hidden())
    print(sent, class_sent)
    print(soft.view(1,-1), class_sent)


epoch, total loss, average loss, duration
0 69.53411465883255 0.695341146588 0:00:00.399789
1 69.475761115551 0.695049378872 0:00:00.391298
2 69.42029881477356 0.694767248631 0:00:00.392310
3 69.38069915771484 0.694527184367 0:00:00.394800
4 69.33786725997925 0.694297482014 0:00:00.394162
5 69.28702795505524 0.694059614937 0:00:00.394378
6 69.24094641208649 0.693823879106 0:00:00.393356
7 69.1935384273529 0.693587817252 0:00:00.392247
8 69.14335262775421 0.693348451588 0:00:00.399479
9 69.09582996368408 0.693109436393 0:00:00.411528


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    count = 0
    
    for t in range(len(train_set)):
        sent = train_set[t][0]
        class_sent = train_set[t][1]

        sent2id = get_sent2id(sent)
        sentence = torch.tensor(np.asarray(sent2id), dtype= torch.long)
        class_sent = torch.tensor(np.asarray(class_sent), dtype= torch.long).view(1)

        soft, hidden = model(sentence, model.init_hidden())
        pred = torch.argmax(soft.view(1,-1))
#         print(sent, class_sent)
#         print(pred, soft.view(1,-1), class_sent)

       # if 
        if pred == class_sent:
            count += 1
    
    print('acc_train', count/len(train_set))

In [78]:
with torch.no_grad():
    count = 0
    
    for t in range(len(test_set)):
        sent = test_set[t][0]
        class_sent = test_set[t][1]

        sent2id = get_sent2id(sent)
        sentence = torch.tensor(np.asarray(sent2id), dtype= torch.long)
        class_sent = torch.tensor(np.asarray(class_sent), dtype= torch.long).view(1)

        soft, hidden = model(sentence, model.init_hidden())
        pred = torch.argmax(soft.view(1,-1))
#         print(sent, class_sent)
#         print(pred, soft.view(1,-1), class_sent)

       # if 
        if pred == class_sent:
            count += 1
    
    print('acc_test', count/len(test_set))

acc_test 0.5139826422372228


In [79]:
with torch.no_grad():
    count = 0
    
    for t in range(len(valid_set)):
        sent = valid_set[t][0]
        class_sent = valid_set[t][1]

        sent2id = get_sent2id(sent)
        sentence = torch.tensor(np.asarray(sent2id), dtype= torch.long)
        class_sent = torch.tensor(np.asarray(class_sent), dtype= torch.long).view(1)

        soft, hidden = model(sentence, model.init_hidden())
        pred = torch.argmax(soft.view(1,-1))
#         print(sent, class_sent)
#         print(pred, soft.view(1,-1), class_sent)

       # if 
        if pred == class_sent:
            count += 1
    
    print('acc_valid', count/len(valid_set))

acc_valid 0.4955325232308792
